In [135]:
import matplotlib.pyplot as plt
#plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.patheffects as mpe
import matplotlib
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics


In [136]:
df = pd.read_csv("BBA_PER - Sheet1.csv")

In [137]:
def Classification(CLASS):
    if CLASS >= 0 and CLASS <= 2:
        return 'Poor'
    elif CLASS > 2 and CLASS <= 3:
        return 'Average'
    elif CLASS >3 and CLASS <= 5:
        return 'Good'

In [138]:
df['n_class']= df['P1203'].apply(Classification)

In [139]:
df['Case'].unique()

array(['Driving-3', 'Driving-2', 'Static-1'], dtype=object)

In [140]:
df.columns

Index(['ExpID', 'Case', 'Algo', 'timeslot_sec', 'timeslot_mil',
       'InterIntervalTime', 'AveragePTime', 'AveragePTime_gt100',
       'InterIntervalTime_gt100', 'TPT', 'T_pk', 'T_pk_gt100', '10_per_pk',
       '20_per_pk', '30_per_pk', '40_per_pk', '50_per_pk', '60_per_pk',
       '70_per_pk', '80_per_pk', '90_per_pk', 'Width_Height', 'P1203', 'Stall',
       'Bitrate', 'n_class'],
      dtype='object')

In [141]:
df['n_class']

0          Poor
1          Poor
2          Poor
3          Poor
4          Poor
         ...   
5440    Average
5441    Average
5442       Poor
5443       Poor
5444       Poor
Name: n_class, Length: 5445, dtype: object

In [142]:
# df =df[(df['Case']=='Static-1')]

In [147]:
df =df[(df['Case']=='Driving-2')]

In [148]:
# df =df[(df['Algo']=='bba')| (df['Algo']=='elastic')| (df['Algo']=='conventional')]

In [149]:
df.shape[0]

1988

In [150]:
df['n_class'].unique()

array(['Poor', 'Average', 'Good'], dtype=object)

In [151]:
df.isnull().values.any()

False

In [152]:
nan_cols = [i for i in df.columns if df[i].isnull().any()]

In [153]:
nan_cols

[]

In [154]:
# df['Min_gt100']= df['Min_gt100'].fillna(value=df['Min_gt100'].mean())

In [155]:
# df['Max_gt100']= df['Max_gt100'].fillna(value=df['Max_gt100'].mean())

In [156]:
# df.loc[:, (df == 0).all()]


In [157]:
from sklearn import preprocessing


In [158]:
X = df[['InterIntervalTime', 'AveragePTime', 'AveragePTime_gt100',
       'InterIntervalTime_gt100','TPT', 'T_pk', 'T_pk_gt100', '10_per_pk',
       '20_per_pk', '30_per_pk', '40_per_pk', '50_per_pk', '60_per_pk',
       '70_per_pk', '80_per_pk', '90_per_pk']]
y = df['n_class']

In [159]:
X.columns

Index(['InterIntervalTime', 'AveragePTime', 'AveragePTime_gt100',
       'InterIntervalTime_gt100', 'TPT', 'T_pk', 'T_pk_gt100', '10_per_pk',
       '20_per_pk', '30_per_pk', '40_per_pk', '50_per_pk', '60_per_pk',
       '70_per_pk', '80_per_pk', '90_per_pk'],
      dtype='object')

In [160]:
# X=preprocessing.normalize(X)
X= preprocessing.scale(X)

In [161]:
# y = df.iloc[:, 23]

In [162]:
y

393        Poor
394        Poor
395        Poor
396     Average
397     Average
         ...   
5270       Good
5271       Good
5272       Good
5273       Good
5274       Good
Name: n_class, Length: 1988, dtype: object

In [163]:
y.unique()

array(['Poor', 'Average', 'Good'], dtype=object)

In [164]:
X.shape[0]

1988

In [165]:
sm = SMOTE()
X, y = sm.fit_sample(X, y)

In [166]:
X.shape[0]

5001

In [167]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [168]:
dummy_y

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [169]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(50, input_dim=16,kernel_initializer = 'uniform',  activation='relu'))
    model.add(Dense(150,kernel_initializer = 'uniform',  activation='relu'))
    model.add(Dense(250,kernel_initializer = 'uniform',  activation='relu'))

    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



In [170]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=500, batch_size=128, verbose=0)
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 97.70% (0.50%)


In [171]:
print(results)

[0.96903098 0.98100001 0.98299998 0.97799999 0.97399998]


In [172]:
# estimator.fit(X_train, y_train)
# predictions = estimator.predict(X_test)
#print(predictions)

In [173]:
# print(encoder.inverse_transform(predictions))

In [174]:
X_train, X_test, Y_train, Y_test = train_test_split(X, dummy_y, test_size=0.3)
estimator.fit(X_train, Y_train)
predictions = estimator.predict(X_test)

In [175]:
print(predictions[0])
print(encoder.inverse_transform(predictions))

2
['Poor' 'Average' 'Average' ... 'Good' 'Good' 'Average']


In [176]:
Y_test[0]

array([0., 0., 1.], dtype=float32)

In [177]:
import numpy as np
rounded_labels=np.argmax(Y_test, axis=1)
rounded_labels[3]
# 2

0

In [178]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(rounded_labels, predictions)
cm

array([[495,   5,   4],
       [ 14, 465,   4],
       [ 10,   0, 504]])

In [179]:
print(classification_report(rounded_labels, predictions))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97       504
           1       0.99      0.96      0.98       483
           2       0.98      0.98      0.98       514

    accuracy                           0.98      1501
   macro avg       0.98      0.98      0.98      1501
weighted avg       0.98      0.98      0.98      1501



In [180]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
accuracy_score(rounded_labels, predictions)
print('Accuracy is ' + str(round(accuracy_score(rounded_labels, predictions),2)))

Accuracy is 0.98


In [181]:
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

In [182]:
cmn= np.around(cm / cm.astype(np.float).sum(axis=1, keepdims=True),2) 

In [183]:
cmn

array([[0.98, 0.01, 0.01],
       [0.03, 0.96, 0.01],
       [0.02, 0.  , 0.98]])